# 사후분석 (post hoc)

## #01. 사후분석 개요
ANOVA 검증결과 유의미하다는 결론을 얻었을 떄 구체적으로 어떤 수준에서 평균차이가 나는지 검증하는 방법

연구자의 사전가설없이 ANOVA를 시행한 경우 탐색적으로 평균차이가 나는 수준을 살펴보기위해 시행하는 방법

조합가능한 모든 쌍에 대해 비교를 하므로 과잉검증으로 인한 FWER 증가

### FWER ( 가족오류율 )

가설 검정에서 여러개의 가설을 동시에 비교하는 경우, 각 가설을 독립적으로 검정할 때 발생하는 오류 확률이 증가한다.
FWER는 이러한 가설 중 어느 하나라도 잘못 기각되는 발생할 확률

가설검정을 많이 할 수록 FWER은 증가

### 대표적인 사후분석 방법

유의수준을 보정하여 FWER을 0.05로 고정시킴

- 봉페로니 교정 (일반적으로 널리 사용됨)
- 투키의 HSD (일반적으로 널리 사용됨)
- 피셔의 LSD : 실제로 보정을 하지 않는 방법이므로 사용하지 않음.
- 셰페의 방법 : 지나치게 보수적이어서 사용하지 않음

## #02. 작업 준비
### 패키지 가져오기 

In [ ]:
import pandas as pd

# 분산분석을 위한 라이브러리
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# 사후분석을 위한 라이브러리
from statsmodels.sandbox.stats.multicomp import MultiComparison
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_ind

# helper 참조
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import normality_test, equal_variance_test, independence_test, all_test

## #03. 예제 (1)
품종별 나무 무게 조사 데이터
### 데이터 가져오기


In [4]:
df = pd.read_excel("https://data.hossam.kr/E02/tree_weight.xlsx")
df

,weight,group
0,4.17,A
1,5.58,A
2,5.18,A
3,6.11,A
4,4.50,A
5,4.61,A
6,5.17,A
7,4.53,A
8,5.33,A
9,5.14,A


### 데이터 전처리


In [5]:
df2 = df.copy()
df2['group']=df2['group'].map({"A":0,"B":1,"C":2})
df2

,weight,group
0,4.17,0
1,5.58,0
2,5.18,0
3,6.11,0
4,4.50,0
5,4.61,0
6,5.17,0
7,4.53,0
8,5.33,0
9,5.14,0


### 일원분산분석(전)

정규성, 등분산성, 독립성

In [7]:
all_test(df2['group'],df2['weight'])

Statistic       p-value  Result
Condition Test      Field                                           
정규성       Shapiro   group             0.795503  5.434527e-05   False
                    weight            0.982683  8.915055e-01    True
          normal    group            17.211505  1.830498e-04   False
                    weight            0.568335  7.526404e-01    True
          k-s_2samp group vs weight   1.000000  1.691123e-17   False
                    weight vs group   1.000000  1.691123e-17   False
등분산성      Bartlett  group vs weight   0.812217  3.674655e-01    True
          Fligner   group vs weight   1.419281  2.335219e-01    True
          Levene    group vs weight   0.849158  3.606080e-01    True
독립성       Chi2      group vs weight  15.905074  9.765907e-01    True

### 일원분산분석


In [8]:
model = ols("weight ~ C(group)",data=df2).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(group),2.0,3.76634,1.883170,4.846088,0.01591
Residual,27.0,10.49209,0.388596,NaN,NaN


#### 결과해석

1. p값이 0.05보다 작기 떄문에 그룹에 따른 무게의 평균의 차이가 통계쩍으로 유의하다


### 사후분석
#### 본페로니 교정

- Bonferroni correction
- 모든 집단을 짝지어 t-test
- FWER이 중간정도
 > pval_corr 값이 0.05보다 작은 항목만 통계적으로 유의함

In [15]:
### MultiComparison(결과값,조건값)

comp = MultiComparison(df2['weight'],df2['group'])
result = comp.allpairtest(ttest_ind,method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
0,1,1.1913,0.249,0.7471,False
0,2,-2.134,0.0469,0.1406,False
1,2,-3.0101,0.0075,0.0226,True


### 결과해석
- pval을 확인한 결과 group에 있는 집단을 쌍을 지어 평균차이를 검정한결과 
1. 0과 1은 평균차이가 없다
2. 0과1, 1과2는 평균차이가 있따 

- pval_corr을 분석한 결과
1. 0.05보다 낮은게 1과2의 평균 차이를 검정한거 밖에 없음 : 통계적으로 유의한 차이가 난다 

### 투키의 HSD

FWER이 중간정도


In [17]:
hsd = pairwise_tukeyhsd(df2['weight'],df2['group'],alpha=0.05)

hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
0,1,-0.371,0.3909,-1.0622,0.3202,False
0,2,0.494,0.198,-0.1972,1.1852,False
1,2,0.865,0.012,0.1738,1.5562,True


- meandiff : 각 그룹간 평균의 차이
- p-adj : 그룹간 차이에 따른 p값
- lower & upper : 평균차이에 대한 신뢰구간
- reject : 가설의 채택여부

## 예제(2) 
독극물 실험 데이터

Time	동물의 생존시간

poison	사용된 독극물 종류

treat	사용되는 치료 유형
### 데이터 가져오기

In [21]:
df = pd.read_excel("https://data.hossam.kr/E02/poisons.xlsx")
df

,time,poison,treat
0,0.31,1,A
1,0.45,1,A
2,0.46,1,A
3,0.43,1,A
4,0.36,2,A
5,0.29,2,A
6,0.40,2,A
7,0.23,2,A
8,0.22,3,A
9,0.21,3,A


#### 데이터 전처리 



In [22]:
df2= df.copy()
df2['treat']=df2['treat'].map({"A":1,"B":2,"C":3,"D":4})
df2.head()

,time,poison,treat
0,0.31,1,1
1,0.45,1,1
2,0.46,1,1
3,0.43,1,1
4,0.36,2,1


In [24]:
df2.dtypes

time      float64
poison      int64
treat       int64
dtype: object

### 이원분산분석

정규성, 등분산성, 독립성 검정 생략

In [25]:
model = ols("time ~ C(poison)*C(treat)",data=df2).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(poison),2.0,1.033012,0.516506,23.221737,3.331440e-07
C(treat),3.0,0.921206,0.307069,13.805582,3.777331e-06
C(poison):C(treat),6.0,0.250138,0.041690,1.874333,1.122506e-01
Residual,36.0,0.800725,0.022242,NaN,NaN


> 교호작용 없음, poison이랑 treat은 평균차이 존재 

### 사후분석 Bonferoni

poison에 대한 본페로니 교정

In [27]:
comp = MultiComparison(data=df2['time'],groups=df2['poison'])
result = comp.allpairtest(ttest_ind,method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
1,2,0.8189,0.4193,1.0,False
1,3,6.2474,0.0,0.0,True
2,3,3.6234,0.0011,0.0032,True


treat에 대한 본페로니 교정


In [30]:
comp = MultiComparison(df2['time'],df2['treat'])
result = comp.allpairtest(ttest_ind,method='bonf')
result[0]

group1,group2,stat,pval,pval_corr,reject
1,2,-3.7291,0.0012,0.007,True
1,3,-1.3855,0.1798,1.0,False
1,4,-3.1478,0.0047,0.028,True
2,3,2.7215,0.0125,0.0748,False
2,4,1.27,0.2174,1.0,False
3,4,-1.7796,0.089,0.5338,False


### 사후분석 Tukey

poison에 대한 사후분석

In [31]:
hsd = pairwise_tukeyhsd(df2['time'],df2['poison'],alpha=0.05)
hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
1,2,-0.0731,0.5882,-0.2525,0.1063,False
1,3,-0.3412,0.0001,-0.5206,-0.1619,True
2,3,-0.2681,0.0021,-0.4475,-0.0887,True


In [32]:
hsd = pairwise_tukeyhsd(df2['time'],df2['treat'],alpha=0.05)
hsd.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
1,2,0.3625,0.001,0.1253,0.5997,True
1,3,0.0783,0.8143,-0.1589,0.3156,False
1,4,0.22,0.0778,-0.0172,0.4572,False
2,3,-0.2842,0.0132,-0.5214,-0.0469,True
2,4,-0.1425,0.387,-0.3797,0.0947,False
3,4,0.1417,0.3922,-0.0956,0.3789,False
